**Image Processing Review** <br>
Melissa

Just review images and how many have been processed (i.e., made subtractions).
Which images/exposures haven't been processed? Which might have failed to process?

In [ ]:
import psycopg2
import psycopg2.extras
import getpass

import numpy as np
from copy import deepcopy

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

In [ ]:
### Get from the exposures table
query = "SELECT id, ra, dec, filename, mjd, filter, proposalid FROM exposures"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
exp_id  = np.asarray( results[0], dtype='int' )
exp_ra  = np.asarray( results[1], dtype='float' )
exp_dec = np.asarray( results[2], dtype='float' )
exp_fnm = np.asarray( results[3], dtype='str' )
exp_mjd = np.asarray( results[4], dtype='float' )
exp_fil = np.asarray( results[5], dtype='str' )
exp_pid = np.asarray( results[6], dtype='str' )
del query, results

In [ ]:
exp_fbs = deepcopy(exp_fnm)
for i,fnm in enumerate(exp_fnm):
    exp_fbs[i] = fnm.split('.')[0]
print(exp_fnm[0],exp_fbs[0])

In [ ]:
### Get from the subtractions table
query = "SELECT id, ra, dec, lmt_mg, seeing, skysig, magzp, "+ \
        "ccdnum, image_id, exposure_id FROM subtractions"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
sub_id  = np.asarray( results[0], dtype='int' )
sub_ra  = np.asarray( results[1], dtype='float' )
sub_dec = np.asarray( results[2], dtype='float' )
sub_lmg = np.asarray( results[3], dtype='float' )
sub_see = np.asarray( results[4], dtype='float' )
sub_sks = np.asarray( results[5], dtype='float' )
sub_mzp = np.asarray( results[6], dtype='float' )
sub_ccd = np.asarray( results[7], dtype='int' )
sub_iid = np.asarray( results[8], dtype='int' )
sub_eid = np.asarray( results[9], dtype='int' )
del query, results

In [ ]:
print(np.min(exp_id),np.max(exp_id))
print(np.min(sub_iid),np.max(sub_iid))
print(np.min(sub_eid),np.max(sub_eid))

In [ ]:
### Get from the file made from investigation into raw data files via NOIRLab DataLab
fnm = 'ddf_image_data.txt'
arch_fbase   = np.loadtxt( fnm, dtype='str', usecols=(0) )
arch_ra      = np.loadtxt( fnm, dtype='float', usecols=(1) )
arch_dec     = np.loadtxt( fnm, dtype='float', usecols=(2) )
arch_mjd     = np.loadtxt( fnm, dtype='float', usecols=(3) )
arch_filt    = np.loadtxt( fnm, dtype='str', usecols=(4) )
arch_airm    = np.loadtxt( fnm, dtype='float', usecols=(5) )
arch_obj     = np.loadtxt( fnm, dtype='str', usecols=(6) )
arch_expt    = np.loadtxt( fnm, dtype='float', usecols=(7) )
arch_seqid   = np.loadtxt( fnm, dtype='str', usecols=(8) )
arch_class   = np.loadtxt( fnm, dtype='str', usecols=(9) )
arch_ftype   = np.loadtxt( fnm, dtype='str', usecols=(10) )
arch_moonsep = np.loadtxt( fnm, dtype='float', usecols=(11) )
arch_moonill = np.loadtxt( fnm, dtype='float', usecols=(12) )
arch_mpval   = np.loadtxt( fnm, dtype='float', usecols=(13) )

In [ ]:
### For all the archive raw images, are any unmatched to exposures table?
exp_archid = np.zeros( len(exp_id), dtype='int' )

nomatch = 0
nomatch_class = []
onematch = 0
multimatch = 0

for i,fbase in enumerate(arch_fbase):
    tx = np.where( fbase == exp_fbs )[0]
    if len(tx) == 0:
        nomatch += 1
        nomatch_class.append( arch_class[i] )
    elif len(tx) == 1:
        onematch += 1
        exp_archid[tx] = i
    elif len(tx) > 1:
        multimatch += 1
        print('Multiple matches for ',fbase, exp_fbs[tx])
    del tx
print( 'Number of archive images with mutiple matches to exposure', multimatch )

print(' ')
print( 'Number of archive images with one match to exposure ', onematch )

print(' ')
print( 'Number of archive images with no matches to exposure ', nomatch )
tmps = np.asarray( nomatch_class, dtype='str')
utmps = np.unique( np.sort( tmps ) )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp, len(tx))
    del tx

print(' ')
tx = np.where( exp_archid == 0 )[0]
print('Number of exposures without an archive match: ',len(tx))
del tx

del nomatch,multimatch,nomatch_class,tmps,utmps

In [ ]:
### For all in the exposure table, how many do / do not have a subtraction?

exp_nsubs = np.zeros( len(exp_id), dtype='int' )
for i,expid in enumerate(exp_id):
    tx = np.where( sub_eid == expid )[0]
    exp_nsubs[i] = len(tx)
    del tx

print('Number of subtractions; number of exposures with N subtractions.')
tmps = np.sort(exp_nsubs)
utmps = np.unique(tmps)
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx

In [ ]:
print('What kind of exposures do not have subtractions?')
tx = np.where( exp_nsubs == 0 )[0]
nosub_ra  = np.floor( exp_ra[tx] )
nosub_fil = exp_fil[tx]

tmps = np.sort( nosub_ra )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))

tmps = np.sort( nosub_fil )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))

print('Mostly COSMOS i-band that do not have subs.')

In [ ]:
### Double check on ELAIS processing
x = np.where( arch_class == 'ELAIS' )[0]
for i in x:
    # find matching exposure to this filename base
    tx = np.where( arch_fbase[i] == exp_fbs )[0]
    # find all matching subtractions for the exposure
    sx = np.where( sub_eid == exp_id[tx] )[0]
    if len(tx) == 1:
        print(i,arch_class[i],exp_fbs[tx],len(sx))